# MCP Multi-Agent Example: Multiagent One MCP Server.
This Example Demonstrates a multiagent Orchestration to interact with a filesystem MCP server in order to perform code Analysis. 

## Setup 
- install  ```ag2[openai,mcp]```
- import Agent classes
- import sse, stdio and streamable-http client methods to connect to server via different transport methods.

In [ ]:
! pip3 install -q -U "ag2[openai,mcp]"

In [ ]:
# import necessary agent and client libraries
import os
from datetime import timedelta

import dotenv
from mcp.client.session import ClientSession
from mcp.client.sse import sse_client
from mcp.client.stdio import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client

from autogen import LLMConfig
from autogen.agentchat import AssistantAgent, UserProxyAgent, initiate_group_chat
from autogen.agentchat.group import (
    AgentTarget,
    ReplyResult,
)
from autogen.agentchat.group.patterns import AutoPattern
from autogen.mcp import create_toolkit

dotenv.load_dotenv()
# Only needed for Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

Create an LLM Config

In [ ]:
# create a llm config
llm_config = LLMConfig(model="o4-mini", api_type="openai", api_key=os.getenv("OPENAI_API_KEY"))

### Initialize Agents and orchestrate the Autopattern
- analysis agent : will select the tool to execute and perform analysis on file contents.
- reporter agent : generate reports based on the analysis
- userporxy agent: responsible for tool execution.

In [ ]:
from autogen.agentchat.conversable_agent import ConversableAgent

with llm_config:
    # Create a Analysis Agent
    analysis_agent = ConversableAgent(
        name="analysis_agent",
        system_message="You are a analysis agent that can perform analysis on the content of the file and return the result.",
        human_input_mode="NEVER",
    )
    # Create a report writer Agent
    reporter = ConversableAgent(
        name="report_writer",
        system_message="You are a reporter agent that can report the result of the analysis.",
        human_input_mode="NEVER",
    )
    pass

user_proxy = UserProxyAgent(
    name="Admin",
    human_input_mode="NEVER",
    code_execution_config=False,
    llm_config=llm_config,
)
# define orchestrate a pattern
pattern = AutoPattern(
    initial_agent=analysis_agent,
    agents=[user_proxy, analysis_agent, reporter],
    group_manager_args={"llm_config": llm_config},
)

## MCP Server Client tool with sse transport
- this tool is an example of how an multiagent orcestration can interact with a mcp server with sse transport


In [ ]:
# run mcp ag2 client
tool_prompt = """
use mcp tools to List files and directories under CONTEXT_PATH/relative_path and read the content of the file
args:
query: str (initial message to the agent)
"""
MCP_SERVER_URL = os.getenv("MCP_SERVER_URL")


@user_proxy.register_for_execution(description=tool_prompt)
@analysis_agent.register_for_llm(description=tool_prompt)
async def run_mcp_ag2_client_with_sse_client(query: str):
    async with sse_client(url="http://127.0.0.1:8000/sse") as (read, write), ClientSession(read, write) as session:
        await session.initialize()
        # create a MCP toolkit with create_toolkit method with MCP server session as param
        toolkit = await create_toolkit(session=session)
        agent = AssistantAgent(
            name="assistant",
            llm_config=llm_config,
            human_input_mode="NEVER",
        )
        toolkit.register_for_llm(agent)
        # Make a request using the MCP tool
        result = await agent.a_run(
            message=query,
            tools=toolkit.tools,
            max_turns=2,
        )
        res = await result.process()
        return ReplyResult(message=str(res), target=AgentTarget(analysis_agent))

In [ ]:
# initiate a group chat with the pattern
result, context_variables, last_agent = initiate_group_chat(
    messages="search wikipedia mcp the term 'upcoming movies 2026'",
    max_rounds=3,
    pattern=pattern,
)

## MCP Client with streamable-http transport
- This tool is an example of how an multiagent orchestration can help with streamable-http transport

In [ ]:
MCP_SERVER_URL = os.getenv("MCP_SERVER_URL")
tool_prompt = """
use mcp tools to List files and directories under CONTEXT_PATH/relative_path and read the content of the file
args:
query: str (detail task for mcp server to execute)
"""


@user_proxy.register_for_execution(description=tool_prompt)
@analysis_agent.register_for_llm(description=tool_prompt)
async def run_mcp_ag2_client_with_streamable_http_client(query: str):
    async with (
        streamablehttp_client(MCP_SERVER_URL) as (
            read_stream,
            write_stream,
            _,
        ),
        ClientSession(read_stream, write_stream) as session,
    ):
        await session.initialize()
        # create a MCP toolkit with create_toolkit method with MCP server session as param
        toolkit = await create_toolkit(session=session)
        agent = AssistantAgent(
            name="assistant",
            llm_config=llm_config,
            human_input_mode="NEVER",
        )
        toolkit.register_for_llm(agent)
        # Make a request using the MCP tool
        result = await agent.a_run(
            message=query,
            tools=toolkit.tools,
            max_turns=2,
        )
        res = await result.process()
        return ReplyResult(message=str(res), target=AgentTarget(analysis_agent))

In [ ]:
# initiate a group chat with the pattern
result, context_variables, last_agent = initiate_group_chat(
    messages="analyze the content of the file text.txt",
    max_rounds=5,
    pattern=pattern,
)

## MCP Client with Stdio transport
- This tool is an example of how an multiagent orchestration can help with stdio transport

In [ ]:
MCP_SERVER_PATH = os.getenv("MCP_SERVER_PATH")
tool_prompt = """
"use mcp tools to List files and directories under CONTEXT_PATH/relative_path and read the content of the file
args:
query: str (initial message to the agent)
"""


# Create server parameters for stdio connection
@user_proxy.register_for_execution(description=tool_prompt)
@analysis_agent.register_for_llm(description=tool_prompt)
async def run_mcp_ag2_client_with_stdio_client(query: str):
    server_params = StdioServerParameters(
        command="python",  # The command to run the server
        args=[
            MCP_SERVER_PATH,
            "stdio",
        ],  # Path to server script and transport mode
    )
    async with (
        stdio_client(server_params) as (read, write),
        ClientSession(read, write, read_timeout_seconds=timedelta(seconds=30)) as session,
    ):
        # Initialize the connection
        await session.initialize()
        toolkit = await create_toolkit(session=session)
        agent = AssistantAgent(
            name="assistant",
            llm_config=llm_config,
            human_input_mode="NEVER",
        )
        toolkit.register_for_llm(agent)
        # Make a request using the MCP tool
        result = await agent.a_run(
            message=query,
            tools=toolkit.tools,
            max_turns=2,
        )
        res = await result.process()
        return ReplyResult(message=str(res), target=AgentTarget(analysis_agent))

In [ ]:
# initiate a group chat with the pattern
result, context_variables, last_agent = initiate_group_chat(
    messages="analyze the content of the file text.txt",
    max_rounds=5,
    pattern=pattern,
)